<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignmnet 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from tensorflow.keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 1us/step


In [4]:
tf.__version__

'1.14.0'

In [6]:
tf

<module 'tensorflow' from '/usr/local/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/__init__.py'>

In [8]:
X_train.shape

(404, 13)

In [9]:
y_train

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

In [25]:
X_train.shape

(404, 13)

In [26]:
y_train.shape

(404,)

In [40]:
model = Sequential()
model.add(Dense(10,input_dim=13,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,))

In [44]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(X_train,y_train,epochs=150)

Epoch 1/150
404/404 [==============================] - 0s 220us/sample - loss: 33.1498 - acc: 0.0000e+00
Epoch 2/150
404/404 [==============================] - 0s 29us/sample - loss: 30.7596 - acc: 0.0000e+00
Epoch 3/150
404/404 [==============================] - 0s 29us/sample - loss: 30.2004 - acc: 0.0000e+00
Epoch 4/150
404/404 [==============================] - 0s 29us/sample - loss: 30.2527 - acc: 0.0000e+00
Epoch 5/150
404/404 [==============================] - 0s 29us/sample - loss: 30.5386 - acc: 0.0000e+00
Epoch 6/150
404/404 [==============================] - 0s 27us/sample - loss: 30.9344 - acc: 0.0000e+00
Epoch 7/150
404/404 [==============================] - 0s 30us/sample - loss: 30.2039 - acc: 0.0000e+00
Epoch 8/150
404/404 [==============================] - 0s 28us/sample - loss: 30.6512 - acc: 0.0000e+00
Epoch 9/150
404/404 [==============================] - 0s 28us/sample - loss: 31.3870 - acc: 0.0000e+00
Epoch 10/150
404/404 [==============================] - 0s 26us

In [43]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 10)                140       
_________________________________________________________________
dense_28 (Dense)             (None, 20)                220       
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 21        
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [ ]:
##### Your Code Here #####

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.